# Contest

Оригинальное задание доступно по ссылке: https://stepik.org/lesson/226979/step/1?unit=199528

Задача нам уже знакома - нужно предсказать, сможет ли пользователь успешно закончить онлайн курс Анализ данных в R.

Мы будем считать, что пользователь успешно закончил курс, если он правильно решил больше 40 практических заданий.

В данных:

submission_data_test.csv

events_data_test.csv

хранится информация о решениях и действиях для 6184 студентов за первые два дня прохождения курса. Это 6184 студентов, которые проходили курс в период с мая 2018 по январь 2019. Подробное описание переменных смотри в этом шаге.  

Используя данные о первых двух днях активности на курсе вам нужно предсказать, наберет ли пользователь более 40 баллов на курсе или нет.

В этих данных, вам доступны только первые дня активности студентов для того, чтобы сделать предсказание. На самом деле, используя эти данные, вы уже можете сделать прогноз. Например, если пользователь за первые два дня набрал 40 баллов, скорее всего он наберет более 40 баллов в дальнейшем. Чтобы подкрепить такие гипотезы, вы можете использовать данные, на которые мы исследовали в первых двух модулях курса, где для всех пользователей представлены все данные об их активности на курсе. 

## Результаты

В ходе иследования была использована модель обучения Random Forest Classifier.
Достигнутый результат после обучения - 89% точности ухода пользователя по его активности за первые  дня пребывания на курсе.

## Ход исследования

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.__version__
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [292]:
sns.set(rc={'figure.figsize':(9,6)})
events_data= pd.read_csv('../data_compet/event_data_train.csv')
submissions_events_data= pd.read_csv('../ML.Stepic/submissions_data_train.csv')

In [293]:
def preprocess_timestamp_cols(data):
    """ 
    Parameters
    ----------
    data : pd.DataFrame
        данные с действиями пользователя
    """
    data['date'] = pd.to_datetime(data.timestamp, unit='s')
    data['day'] = data.date.dt.date
    data['weekday'] =  data['date'].dt.weekday_name
    return data

 ### Добавили дату и день в нормальном формате

In [294]:
events_data.head(2)

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632


## Фича сложность степа

In [295]:
threshold = 2*24*60*60
last_timestemp =  preprocess_timestamp_cols(submissions_events_data).sort_values(by='timestamp',ascending = True)\
                        .drop_duplicates('user_id')\
                        .drop(['step_id','submission_status','date', 'day','weekday'],axis =1)
last_timestemp['last_timestamp'] = last_timestemp['timestamp'] + threshold
last_timestemp.rename(columns = {'timestamp' : 'min_timestamp'},inplace = True)
last_timestemp = last_timestemp.merge(submissions_events_data, how='inner',on='user_id')
last_timestemp['date_last'] = pd.to_datetime(last_timestemp.last_timestamp, unit='s')
submissions_2_days = last_timestemp[last_timestemp['timestamp'] <= last_timestemp['last_timestamp']]
submissions_2_days.groupby('user_id')[['day']].nunique().head()

,day
user_id,
2,1
3,1
5,1
8,1
14,1


In [296]:
def step_diff(data):
    
    """
    Рассчитывает сложность степов, которые прошел пользователь
    data - данные о сабмитах на первые два дня
    возвращает Sub_step_diff - фичу с юзер_ид и оценкой сложности степов, которые прошел пользователь
    """
    
    
    
    step_difficulty = data.pivot_table(index='step_id',
                                            columns='submission_status',
                                            values='user_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()
    
    step_difficulty['difficulty'] = step_difficulty['wrong'] / step_difficulty['correct']
    Sub_step_diff = data.merge(step_difficulty, on='step_id',  how='outer')
    Sub_step_diff = Sub_step_diff[['user_id', 'difficulty']].groupby('user_id', as_index = False).sum()
    return Sub_step_diff
    

In [297]:
Sub_step_diff = step_diff(submissions_2_days)

## Students activity

In [298]:
def create_user_data(events_data, submissions_events_data):
    """ создать таблицу с данными по каждому пользователю
    Parameters
    ----------
    events_data : pd.DataFrame
        данные с действиями пользователя
    submissions_data : pd.DataFrame
        данные самбитов практики
    """
    users_data = events_data.groupby('user_id', as_index=False) \
        .agg({'timestamp': 'max'}).rename(columns={'timestamp': 'last_timestamp'})

    # попытки сдачи практики пользователя
    users_scores = submissions_events_data.pivot_table(index='user_id',
                                                columns='submission_status',
                                                values='step_id',
                                                aggfunc='count',
                                                fill_value=0).reset_index()
    users_data = users_data.merge(users_scores, on='user_id', how='outer')
    users_data = users_data.fillna(0)
    
    # сложность кадой задачи
    users_data = users_data.merge(Sub_step_diff, on='user_id', how='outer')
    users_data = users_data.fillna(0)

    # колво разных событий пользователя по урокам
    users_events_data = events_data.pivot_table(index='user_id',
                                                columns='action',
                                                values='step_id',
                                                aggfunc='count',
                                                fill_value=0).reset_index()
    users_data = users_data.merge(users_events_data, how='outer')

    # колво дней на курсе
    users_days = events_data.groupby('user_id').day.nunique().to_frame().reset_index()
    users_data = users_data.merge(users_days, how='outer')
    
    #пройдет ли курс
    users_data['is_gone_user_passed'] = users_data.passed > 40
    users_data['is_gone_user_correct'] = users_data.correct >40
    
    
    assert users_data.user_id.nunique() == events_data.user_id.nunique()
    
    return users_data

In [299]:
submissions_events_data.groupby(['user_id','submission_status','weekday']).nunique().loc[:100,:].head()

step_id  timestamp  submission_status  \
user_id submission_status weekday                                            
2       correct           Wednesday        2          2                  1   
3       correct           Monday           4          4                  1   
                          Saturday        10         10                  1   
                          Sunday           6          6                  1   
                          Thursday         9          9                  1   

                                     user_id  date  day  weekday  
user_id submission_status weekday                                 
2       correct           Wednesday        1     2    1        1  
3       correct           Monday           1     4    1        1  
                          Saturday         1    10    1        1  
                          Sunday           1     6    1        1  
                          Thursday         1     9    2        1

In [300]:
def truncate_data_by_nday(data, n_day):
    """ Взять события из n_day первых дней по каждому пользователю 
    
        Parameters
        ----------
        data: pandas.DataFrame
            действия студентов со степами или практикой
        n_day : int
            размер тестовой выборки
    """
    users_min_time = data.groupby('user_id', as_index=False).agg({'timestamp': 'min'}).rename(
        {'timestamp': 'min_timestamp'}, axis=1)
    users_min_time['min_timestamp'] += 60 * 60 * 24 * n_day

    events_data_d = pd.merge(data, users_min_time, how='inner', on='user_id')
    cond = events_data_d['timestamp'] <= events_data_d['min_timestamp']
    events_data_d = events_data_d[cond]

    assert events_data_d.user_id.nunique() == data.user_id.nunique()
    return events_data_d.drop(['min_timestamp'], axis=1)

In [301]:
import collections
def safe_drop_cols_df(df, drop_cols):
    """ Удаление столбцов из датафрейма с проверкой на их существование.
    Изменяет переданный датафрейм
    
    Parameters
    ----------
    df: pandas.DataFrame
        датафрейм
    drop_cols: list of string
        Список столбцов которые нужно удалить
    """

    if isinstance(drop_cols, str) or (not isinstance(drop_cols, collections.Iterable)):
        drop_cols = [drop_cols]
    drop_col_names = np.intersect1d(df.columns, drop_cols)
    df.drop(drop_col_names, axis=1, inplace=True)

In [302]:
def get_x_y(events, submissions, n):
    """" создадим признаки и метку
     
    Parameters
    ----------
    events: pandas.DataFrame
        действия студентов со степами
    submissions: pandas.DataFrame
        действия студентов по практике
    n != 1 если хотим разметить по correct, n=1, если хотим разметить по 'passed'
     """
    DATA_PERIOD_DAYS = 2
        
        
#     for i in events, submissions:
#         preprocess_timestamp_cols(i)
#     for i in events, submissions:    
#         truncate_data_by_nday(i,DATA_PERIOD_DAYS)   
    events_data = preprocess_timestamp_cols(events)
    submissions_data = preprocess_timestamp_cols(submissions)  
    
    events_data_pre = truncate_data_by_nday(events_data,DATA_PERIOD_DAYS)
    submissions_data_pre = truncate_data_by_nday(submissions_data,DATA_PERIOD_DAYS)
    
    
    X = create_user_data(events_data_pre, submissions_data_pre)
    X = X.set_index('user_id')
    
    
    if n == 1:
        y = create_user_data(events_data, submissions_data).is_gone_user_passed
    else:
        y = create_user_data(events_data, submissions_data).is_gone_user_correct

    safe_drop_cols_df(X, ['last_timestamp','is_gone_user_passed', 'is_gone_user_correct'])   
    # после создания признаков и метки порядок следования user_id может не совпадать
    X = X.sort_index()
    y = y.sort_index()
    assert X.shape[0] == y.shape[0]
    return X, y

In [303]:
X,y = get_x_y(events_data, submissions_events_data,1)

In [304]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19234 entries, 1 to 26798
Data columns (total 8 columns):
correct            19234 non-null float64
wrong              19234 non-null float64
difficulty         19234 non-null float64
discovered         19234 non-null int64
passed             19234 non-null int64
started_attempt    19234 non-null int64
viewed             19234 non-null int64
day                19234 non-null int64
dtypes: float64(3), int64(5)
memory usage: 1.3 MB


In [305]:
y.value_counts()

False    14596
True      4638
Name: is_gone_user_passed, dtype: int64

# ОБУЧИМ МОДЕЛЬ

In [306]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

In [93]:
clf = tree.DecisionTreeClassifier(criterion='entropy')

parametrs = {'max_depth' : range(1,20), 
             'min_samples_leaf' : range(10,50,5),
             'min_samples_split' : range(10,50,10)}

search = GridSearchCV(clf,parametrs, cv = 5,verbose = 1, n_jobs =-1)
search.fit(X_train, y_train)

best_tree = search.best_estimator_
best_tree

Fitting 5 folds for each of 608 candidates, totalling 3040 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 2368 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 3040 out of 3040 | elapsed:    9.1s finished


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [94]:
search.best_params_

{'max_depth': 4, 'min_samples_leaf': 10, 'min_samples_split': 10}

In [95]:
best_tree.score(X_train, y_train),best_tree.score(X_test, y_test)

(0.9061772466242434, 0.9050094517958412)

In [96]:
from sklearn.metrics import precision_score, recall_score

y_pred = best_tree.predict(X_test)

recall_score(y_test, y_pred)

0.1206896551724138

In [97]:
precision_score(y_test, y_pred)

0.6470588235294118

In [98]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
pred_proba = best_tree.predict_proba(X_test)
pd.DataFrame(pred_proba).head()

,0,1
0,1.00000,0.00000
1,1.00000,0.00000
2,0.72973,0.27027
3,1.00000,0.00000
4,1.00000,0.00000


In [99]:
roc_score = roc_auc_score(y_test, pred_proba[:, 1])
print('roc на test', roc_score)
# должны получить roc 0.92  +- 0.02

roc на test 0.87319927641656


In [288]:
clf_rf = RandomForestClassifier()

parametrs = {'n_estimators': [10,51,2],'max_depth' : range(10,30,2), 
             'min_samples_leaf' : range(10,51,2),
             'min_samples_split' : range(20,100,2)}

grid_search_cv_clf = GridSearchCV(clf_rf, parametrs, cv = 5,verbose = 1, n_jobs =-1)

grid_search_cv_clf.fit(X_train, y_train)

grid_search_cv_clf.best_params_

best_clf = grid_search_cv_clf.best_estimator_
best_clf

Fitting 5 folds for each of 25200 candidates, totalling 126000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 1300 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 2200 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 3300 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4600 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 6100 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 7800 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 9700 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 11800 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 14100 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 16600 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 19300 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 22200 tasks  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=96,
            min_weight_fraction_leaf=0.0, n_estimators=2, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [289]:
grid_search_cv_clf.best_params_

{'max_depth': 12,
 'min_samples_leaf': 20,
 'min_samples_split': 96,
 'n_estimators': 2}

In [290]:
feature_importances = best_clf.feature_importances_

feature_importances_df = pd.DataFrame({'features' :list(X_train),
                                    'feature_importances': feature_importances})

feature_importances_df.sort_values('feature_importances', ascending=False)

,features,feature_importances
0,correct,0.540795
4,passed,0.313475
2,difficulty,0.040848
5,started_attempt,0.030790
6,viewed,0.026307
3,discovered,0.016533
7,day,0.016397
1,wrong,0.014855


In [307]:
from sklearn.metrics import precision_score, recall_score

y_pred = best_clf.predict(X_test)

recall_score(y_test, y_pred),precision_score(y_test, y_pred)

(0.1503184713375796, 0.9672131147540983)

In [308]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
pred_proba = best_clf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba[:, 1])
print('roc на test', roc_score)
# должны получить roc 0.92  +- 0.02

roc на test 0.9006059487086513


In [309]:
test_event = pd.read_csv('../ML.Stepic/data_compet/events_data_test.csv')
test_submission = pd.read_csv('../ML.Stepic/data_compet/submission_data_test.csv')

In [310]:
# get_x_y(test_event,test_submission,1)

In [311]:
Sub_step_diff = step_diff(preprocess_timestamp_cols(test_submission))

In [313]:
X_test = create_user_data(preprocess_timestamp_cols(test_event),preprocess_timestamp_cols(test_submission)).drop(['is_gone_user_passed', 
                                                            'is_gone_user_correct', 
                                                            'last_timestamp'], 
                                                               axis = 1).set_index('user_id')

In [314]:
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(0)

In [317]:
pred_proba = best_clf.predict_proba(X_test)
submit = pd.DataFrame(pred_proba)
# должны получить roc 0.92  +- 0.02

In [206]:
def create_report(user_ids, preds):
    res = pd.DataFrame(preds[:, np.newaxis], columns=['is_gone'], index=user_ids)
    return res.reset_index()#(submit)

In [318]:
submit['user_id'] = X_test.index
submit['is_gone'] = submit.iloc[:,1]
submit.drop(submit.iloc[:, [0,1]], axis=1, inplace =True)
submit.shape

(6184, 2)

In [319]:
submit.to_csv('submit3.csv')# сделали 3 сабмит

In [216]:
example = pd.read_csv('../ML.Stepic/data_compet/submission_example .csv')
example.head()

,user_id,is_gone
0,12,0.26


In [233]:
X_test

,correct,wrong,difficulty,discovered,passed,started_attempt,viewed,day
user_id,,,,,,,,
4,0.0,0.0,0.000000,1,1,0,1,1
6,0.0,0.0,0.000000,1,1,0,1,1
10,0.0,0.0,0.000000,2,2,0,6,1
12,1.0,0.0,0.014968,11,9,4,14,1
13,29.0,36.0,130.078578,70,70,35,105,2
15,10.0,30.0,60.675658,1,1,0,1,1
19,0.0,0.0,0.000000,1,1,0,1,1
21,24.0,103.0,284.969206,74,68,70,98,2
23,0.0,0.0,0.000000,1,1,0,1,1


In [252]:
test_submission.columns

Index(['step_id', 'timestamp', 'submission_status', 'user_id', 'date', 'day',
       'weekday'],
      dtype='object')

In [253]:
test_data = test_submission.pivot_table(index='user_id',
                                        columns='submission_status',
                                        values='step_id',
                                        aggfunc='count',
                                        fill_value=0).reset_index()



In [283]:
passed_over_40 =test_data[test_data['correct'] >= 30]
passed_over_40

submission_status,user_id,correct,wrong
11,102,34,48
24,186,31,18
30,208,37,30
90,765,31,8
137,1359,36,140
148,1441,31,56
152,1459,30,14
249,2384,38,17
339,3239,34,16
405,3940,33,24


In [285]:
m = list(passed_over_40.user_id)

In [286]:
submit2 = submit.set_index('user_id')
submit2.loc[m, :] = 0.8
# submit1 = submit.set_index('user_id')
# submit1.loc[m, :] = 0.99

## Сохранение результатов

In [287]:
submit2.to_csv('submit2.csv')

In [320]:
submit

,user_id,is_gone
0,4,0.000193
1,6,0.000193
2,10,0.000000
3,12,0.007407
4,13,0.601103
5,15,0.296825
6,19,0.000193
7,21,0.630984
8,23,0.000193
9,35,0.061158
